# CLSS Template Export Process
This notebook is designed to export the CLSS templates and related data to the `export` directory. The exported data can be used for backup, migration, or further analysis.

#### Prerequisites
Before running this notebook, please ensure the following:

1. **Upload Required Files**:
   - Ensure that any necessary input files are uploaded to the `/home` directory in ArcGIS Online/Notebook.
2. **Configuration File**:
   - Ensure that a `config.json` file is present in the `/home` directory. This file should contain your ArcGIS Online (AGO) username and password in the following format:
     ```json
     {
       "AGO_username": "your_username",
       "AGO_password": "your_password"
     }
     ```

#### Steps
1. **Load and Validate Files**:
   - The notebook will check for the presence of necessary files and the `config.json` file in the `/home` directory.
2. **Authenticate with ArcGIS Online**:
   - The notebook will authenticate with ArcGIS Online using the credentials provided in the `config.json` file.
3. **Export Data**:
   - The notebook will export the CLSS templates and related data to the `export` directory.
4. **Save Exported Data**:
   - The exported data will be saved in the `export` directory for further use.

#### Notes
- Ensure that the necessary input files are correctly formatted and contain the required data for exporting.

#### Links
- [DevOps Story #1780](https://dev.azure.com/ghinternational/GHIS/_workitems/edit/1780)


---

**Instructions**: Follow the steps outlined above to ensure all prerequisites are met. Then, run the cells in the notebook sequentially to export the CLSS templates and related data to the `export` directory.

# Import the required libraries


In [1]:
# Standard library imports
import os
import json
# import datetime
from datetime import datetime
import pytz

# Third-party library imports
import pandas as pd
from arcgis.gis import GIS
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Check if /home folder exists, if not create it
home_dir = 'home'

time_zone = 'UTC'  # time zone default is UTC but can be set to any local time zone,e.g. 'US/Eastern'

if not os.path.exists(home_dir):
    os.makedirs(home_dir)

# Check if ../export folder exists, if not create it
export_dir = f'{home_dir}/export'
if not os.path.exists(export_dir):
    os.makedirs(export_dir)

# These are the CLSS ArcGIS online Service Endpoints 
clssServices = [('None', 'None'),  # Example ID for Dev
             ('CLSS Dev', 'ee81054b84fa436bb04860d36845d1fd'),  
             ('CLSS Test', 'f726c24e77d5442c9a9f456eec62ae5d'),  
             ('CLSS Demo', '649091ebf4af4233902e975798d59ef9')]

In [10]:
# Authenticate with ArcGIS Online
try:
    gis = GIS("home")
except Exception as e:
    try:
        # Get username and password from config file
        config_path = f'{home_dir}/config.json'
        if not os.path.isfile(config_path):
            raise FileNotFoundError(f"Config file '{config_path}' does not exist. Please upload 'config.json' to the home directory.")

        with open(config_path) as config_file:
            config = json.load(config_file)

        AGO_username = config['AGO_username']
        AGO_password = config['AGO_password']

        if not AGO_username or not AGO_password:
            raise ValueError("AGO_username and AGO_password must be provided in the config.json file.")
        gis = GIS("https://ghis.maps.arcgis.com/", AGO_username, AGO_password)
    except Exception as e:
        raise ConnectionError(f"Failed to authenticate with ArcGIS Online: {e}")

### Define Functions

In [15]:

# Function to get the related records
def getRelatedName(objId, relateId, outFields):
    try:
        related_table = template_table.query_related_records(
            object_ids=objId, relationship_id=relateId, out_fields=outFields
        )
        return related_table['relatedRecordGroups'][0]['relatedRecords'][0]['attributes']['Name']
    except:
        return "N/A"

# Transform JSON content to DataFrame
def transform_content_to_dataframe(json_data):
    """
    Transforms a nested JSON object into a pandas DataFrame
    with columns: Lifeline, Component, Indicator, Weight.
    """
    data = []
    for lifeline in json_data:
        for component in lifeline.get('componentTemplates', []):
            for indicator in component.get('indicators', []):
                data.append({
                    'Lifeline': lifeline['name'],
                    'Component': component['name'],
                    'Indicator': indicator['name'],
                    'Weight': indicator['weight'],
                })
    return pd.DataFrame(data)

# Export CSV files
def exportCsv(selected_feature_service, selected_template):
    global template_table
    template_table = selected_feature_service.tables[9]

    # Query the selected template
    selectedTemplateFeature = template_table.query(
        where=f"Name = '{selected_template}'", out_fields="*"
    ).features
    dfTemplateInfo = pd.DataFrame([feature.attributes for feature in selectedTemplateFeature])

    # Convert timestamps to human-readable dates
    dfTemplateInfo['CreationDate'] = pd.to_datetime(dfTemplateInfo['CreationDate'], unit='ms').dt.strftime('%d-%m-%Y')
    dfTemplateInfo['EditDate'] = pd.to_datetime(dfTemplateInfo['EditDate'], unit='ms').dt.strftime('%d-%m-%Y')

    # Add related fields
    dfTemplateInfo['HazardName'] = dfTemplateInfo.apply(lambda row: getRelatedName(row['OBJECTID'], 6, "Name"), axis=1)
    dfTemplateInfo['OrganizationName'] = dfTemplateInfo.apply(lambda row: getRelatedName(row['OBJECTID'], 1, "Name"), axis=1)

    # Parse JSON content for indicator details
    try:
        json_content = dfTemplateInfo['Content'].iloc[0]
        json_data = json.loads(json_content)
        dfIndicatorList = transform_content_to_dataframe(json_data)
    except Exception as e:
        print(f"Error parsing JSON data: {e}")
        return

    # Drop unnecessary columns & rename a few others
    dfTemplateInfo.drop(columns=['OBJECTID', 'GlobalID', 'Content', 'HazardID', 'OrganizationID','IsDeleted','Creator','CreationDate','Editor','EditDate'], inplace=True)
    dfTemplateInfo.rename(columns={'Name': 'Template Name', 'Description': 'Template Description'}, inplace=True)
    # Expand dfTemplateInfo to match the number of rows in dfIndicatorList
    dfTemplateInfo_expanded = pd.concat([dfTemplateInfo] * len(dfIndicatorList), ignore_index=True)
    dfCombined = pd.concat([dfTemplateInfo_expanded.reset_index(drop=True), dfIndicatorList.reset_index(drop=True)], axis=1)

    # Generate file paths
    current_datetime = datetime.now(pytz.timezone(time_zone)).strftime('%Y%m%d_%H%M%S_%Z')
    export_dir = './home/export'  # Adjust as needed
    combined_csv_path = f'{export_dir}/{selected_feature_service.title}_{selected_template}_Combined_{current_datetime}.csv'

    # Save the combined DataFrame as a CSV
    dfCombined.to_csv(combined_csv_path, index=False)

    print(f"Successfully exported combined CSV: {combined_csv_path}")

# Function to export the template table
def exportTemplate(selected_feature_service):
    global template_table
    template_table = selected_feature_service.tables[9]
    templateFeatures = template_table.query(where="isDeleted = 0", out_fields=['OrganizationID', 'HazardID', 'Name', 'Description', 'Status', 'Content']).features
    dfTemplate = pd.DataFrame([feature.attributes for feature in templateFeatures])
    # Create a dropdown for selecting the template
    template_choice = widgets.Dropdown(
        options=['None'] + dfTemplate['Name'].tolist(),
        description='Select Template:',
        disabled=False,
    )

    # Handle template selection
    def on_template_selected(change):
        selected_template = change['new']
        print(f"Selected Template: {selected_template}")
        exportCsv(selected_feature_service, selected_template)

    display(template_choice)
    template_choice.observe(on_template_selected, names='value')

# Handle feature service selection
def on_service_selected(change):
    global selected_feature_service
    selected_service_id = change['new']
    print(f"Selected Service ID: {selected_service_id}")
    
    if selected_service_id == 'None':
        print("No feature service selected.")
        return
    
    selected_feature_service = gis.content.get(selected_service_id)
    if selected_feature_service is None:
        print(f"Error: No feature service found with ID {selected_service_id}")
        return
    
    print(f"Selected Feature Service: {selected_feature_service.title}")
    exportTemplate(selected_feature_service)

# Create a dropdown for feature service selection
service_choice = widgets.Dropdown(
    options=clssServices,
    description='Select FS:',
    disabled=False,
)

display(service_choice)
service_choice.observe(on_service_selected, names='value')


Dropdown(description='Select FS:', options=(('None', 'None'), ('CLSS Dev', '4cec9a93384543e0a676e3ad892362bb')…

Dropdown(description='Select Template:', options=('None', 'CLSS System Baseline', 'Template#', 'Drew Template …